In [3]:
# 런타임 애러 방지
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [2]:
!nvidia-smi

Tue May 30 02:00:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.51       Driver Version: 457.51       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   48C    P8    11W / 130W |    606MiB /  6144MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import pandas as pd

train_set = pd.read_csv('data/병합데이터셋-v3.csv', index_col=0) 

# 감정을 정수 라벨로 변경
emotions = {'행복': 0, '불안': 1, '놀람': 2, '슬픔': 3, '분노': 4, '중립': 5 }
train_set['emotion'] = train_set.emotion.map(emotions)

train_set.sample(n=5)

,sentence,emotion
11628,이러나 저러나 집에서 지금 하는 일이 짱이예요.,0
115211,근데 연상녀 좋아하고있더라고요?..,1
112483,친구가 무사히 출산을 했다고 알려왔는데 얼마나 다행인지 몰라.,0
5428,왠만하면 걸치고 있는 코트 벗으시지?,5
132196,자식들이 아무도 나를 도와주지 않아서 슬퍼.,2


In [5]:
# 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import gluonnlp as nlp

#kobert
from kobert_tokenizer import KoBERTTokenizer

# transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 사용시 필요
device = torch.device("cuda")

---

__learning rate = e-5__

In [4]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64    
batch_size = 32 #베이스라인 64
warmup_ratio = 0.1
num_epochs = 8  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5

In [5]:
# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

In [6]:
# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,     
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [7]:
# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류 
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=0)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=0)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수(로딩속도)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=0)

In [8]:
# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [9]:
# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)  #로컬에서는 디폴트(0)으로 수정
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [10]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8971830606460571 train acc 0.15625
epoch 1 batch id 201 loss 1.8041564226150513 train acc 0.14894278606965175
epoch 1 batch id 401 loss 1.785013198852539 train acc 0.17573254364089774
epoch 1 batch id 601 loss 1.5961964130401611 train acc 0.22753743760399334
epoch 1 batch id 801 loss 1.3921833038330078 train acc 0.27071629213483145
epoch 1 batch id 1001 loss 1.238572597503662 train acc 0.3160277222777223
epoch 1 batch id 1201 loss 1.423027753829956 train acc 0.3473667776852623
epoch 1 batch id 1401 loss 1.1432969570159912 train acc 0.37517844396859384
epoch 1 batch id 1601 loss 1.0395294427871704 train acc 0.39789584634603375
epoch 1 batch id 1801 loss 1.2298238277435303 train acc 0.41837867851193783
epoch 1 batch id 2001 loss 1.1632047891616821 train acc 0.4368753123438281
epoch 1 batch id 2201 loss 1.0964641571044922 train acc 0.454253748296229
epoch 1 batch id 2401 loss 1.249979019165039 train acc 0.46889316951270305
epoch 1 batch id 2601 loss 1.21470475196

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 1 test acc 0.6599916141752445


  0%|          | 0/3395 [00:00<?, ?it/s]

...

epoch 7 test acc 0.6868470579198033


  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 8 batch id 1 loss 0.4622437357902527 train acc 0.875
epoch 8 batch id 201 loss 0.7556312680244446 train acc 0.8215174129353234
epoch 8 batch id 401 loss 0.5616306662559509 train acc 0.8144482543640897
epoch 8 batch id 601 loss 0.5034376978874207 train acc 0.8138519134775375
epoch 8 batch id 801 loss 0.6494439840316772 train acc 0.8105493133583022
epoch 8 batch id 1001 loss 0.7208328247070312 train acc 0.8106580919080919
epoch 8 batch id 1201 loss 0.7784857153892517 train acc 0.8110689009159034
epoch 8 batch id 1401 loss 0.5861051678657532 train acc 0.8108940042826552
epoch 8 batch id 1601 loss 0.6852321624755859 train acc 0.8097478138663335
epoch 8 batch id 1801 loss 0.5190677642822266 train acc 0.8090123542476402
epoch 8 batch id 2001 loss 0.8699840307235718 train acc 0.8094234132933533
epoch 8 batch id 2201 loss 0.6189286708831787 train acc 0.8101857110404361
epoch 8 batch id 2401 loss 0.6380034685134888 train acc 0.8110162432319866
epoch 8 batch id 2601 loss 0.6871709823608398

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 8 test acc 0.6872519460234546


학습률: e-5, 에포크: 5 )0.6911824012905208

---

__learingrate = 5e-6__

In [5]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64    
batch_size = 32 #베이스라인 64
warmup_ratio = 0.1
num_epochs = 8  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-6

# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))
    
# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,     
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [6]:
# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류 
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=0)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=0)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수(로딩속도)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=0)

# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)  #로컬에서는 디폴트(0)으로 수정
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [7]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7560423612594604 train acc 0.25
epoch 1 batch id 201 loss 1.8199613094329834 train acc 0.18921019900497513
epoch 1 batch id 401 loss 1.7832614183425903 train acc 0.2214775561097257
epoch 1 batch id 601 loss 1.6579742431640625 train acc 0.2599313643926789
epoch 1 batch id 801 loss 1.55643892288208 train acc 0.2914715980024969
epoch 1 batch id 1001 loss 1.4163613319396973 train acc 0.3179632867132867
epoch 1 batch id 1201 loss 1.5955044031143188 train acc 0.3362822647793505
epoch 1 batch id 1401 loss 1.3921822309494019 train acc 0.3553042469664525
epoch 1 batch id 1601 loss 1.1526474952697754 train acc 0.37441442848219864
epoch 1 batch id 1801 loss 1.3657439947128296 train acc 0.38988756246529704
epoch 1 batch id 2001 loss 1.4022252559661865 train acc 0.40445402298850575
epoch 1 batch id 2201 loss 1.257387638092041 train acc 0.4181763970922308
epoch 1 batch id 2401 loss 1.2686065435409546 train acc 0.43157798833819244
epoch 1 batch id 2601 loss 1.206778287887573

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 1 test acc 0.6482722640446561


  0%|          | 0/3395 [00:00<?, ?it/s]

...

epoch 7 test acc 0.6888346904286372


  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 8 batch id 1 loss 0.6644065380096436 train acc 0.6875
epoch 8 batch id 201 loss 0.758638322353363 train acc 0.7534203980099502
epoch 8 batch id 401 loss 0.5922772884368896 train acc 0.7510910224438903
epoch 8 batch id 601 loss 0.7285424470901489 train acc 0.7514039101497504
epoch 8 batch id 801 loss 0.7126544117927551 train acc 0.7483614232209738
epoch 8 batch id 1001 loss 0.7316358685493469 train acc 0.7499687812187812
epoch 8 batch id 1201 loss 0.9857854247093201 train acc 0.7491933805162365
epoch 8 batch id 1401 loss 0.8305946588516235 train acc 0.7493085296216988
epoch 8 batch id 1601 loss 0.6849831342697144 train acc 0.748145690193629
epoch 8 batch id 1801 loss 0.8749753832817078 train acc 0.7470502498611882
epoch 8 batch id 2001 loss 0.7876958847045898 train acc 0.7469546476761619
epoch 8 batch id 2201 loss 0.7933128476142883 train acc 0.7479696728759655
epoch 8 batch id 2401 loss 0.6795426607131958 train acc 0.7486984589754269
epoch 8 batch id 2601 loss 0.891927182674408 t

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 8 test acc 0.6888714984380601


학습률: 5e-6, 에포크: 6 ) 0.6894748297229477

---

__Learning rate = e-4__

In [4]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64    
batch_size = 32 #베이스라인 64
warmup_ratio = 0.1
num_epochs = 6  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-4

# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))
    
# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,     
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [6]:
# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류 
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=0)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=0)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수(로딩속도)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=0)

# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)  #로컬에서는 디폴트(0)으로 수정
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [7]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.864007830619812 train acc 0.1875
epoch 1 batch id 201 loss 1.5482573509216309 train acc 0.2751865671641791
epoch 1 batch id 401 loss 1.2720866203308105 train acc 0.3870791770573566
epoch 1 batch id 601 loss 1.0586458444595337 train acc 0.45211106489184694
epoch 1 batch id 801 loss 1.0244446992874146 train acc 0.49020755305867664
epoch 1 batch id 1001 loss 0.9156310558319092 train acc 0.5175761738261738
epoch 1 batch id 1201 loss 1.1014080047607422 train acc 0.5339560782681099
epoch 1 batch id 1401 loss 0.9878259897232056 train acc 0.5471538187009279
epoch 1 batch id 1601 loss 0.9179600477218628 train acc 0.5560977514053717
epoch 1 batch id 1801 loss 1.2706944942474365 train acc 0.5627255691282621
epoch 1 batch id 2001 loss 0.9917345643043518 train acc 0.5679660169915043
epoch 1 batch id 2201 loss 0.8570663332939148 train acc 0.5734609268514311
epoch 1 batch id 2401 loss 1.1694954633712769 train acc 0.577741045397751
epoch 1 batch id 2601 loss 0.964412689208984

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 1 test acc 0.656907743124904


  0%|          | 0/3395 [00:00<?, ?it/s]

...

epoch 5 test acc 0.6763343703589901


  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 6 batch id 1 loss 0.3354267477989197 train acc 0.875
epoch 6 batch id 201 loss 0.40598466992378235 train acc 0.8532338308457711
epoch 6 batch id 401 loss 0.3074420988559723 train acc 0.8516988778054863
epoch 6 batch id 601 loss 0.46637555956840515 train acc 0.8545133111480865
epoch 6 batch id 801 loss 0.4471226930618286 train acc 0.8554151061173533
epoch 6 batch id 1001 loss 0.3513924479484558 train acc 0.8585789210789211
epoch 6 batch id 1201 loss 0.3365473747253418 train acc 0.8601165695253955
epoch 6 batch id 1401 loss 0.4722123146057129 train acc 0.8622412562455389
epoch 6 batch id 1601 loss 0.3823358416557312 train acc 0.8629177076826983
epoch 6 batch id 1801 loss 0.6051333546638489 train acc 0.8631836479733481
epoch 6 batch id 2001 loss 0.5618377327919006 train acc 0.8644427786106946
epoch 6 batch id 2201 loss 0.39600956439971924 train acc 0.8659842117219446
epoch 6 batch id 2401 loss 0.24185024201869965 train acc 0.867112661391087
epoch 6 batch id 2601 loss 0.5746057033538

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 6 test acc 0.6754509781328417


In [ ]:
학습률 e-4, 에포크4) 0.6691936165309571

---

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64    
batch_size = 32 #베이스라인 64
warmup_ratio = 0.1
num_epochs = 6  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 0.05

# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))
    
# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,     
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [7]:
# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류 
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=0)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=0)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수(로딩속도)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=0)

# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)  #로컬에서는 디폴트(0)으로 수정
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [8]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8784935474395752 train acc 0.125
epoch 1 batch id 201 loss 1.6341322660446167 train acc 0.2691231343283582
epoch 1 batch id 401 loss 1.7473065853118896 train acc 0.2627805486284289
epoch 1 batch id 601 loss 1.9483098983764648 train acc 0.2621672212978369
epoch 1 batch id 801 loss 1.8602584600448608 train acc 0.25858302122347065
epoch 1 batch id 1001 loss 2.070012331008911 train acc 0.2563998501498502
epoch 1 batch id 1201 loss 3.2637345790863037 train acc 0.2505203996669442
epoch 1 batch id 1401 loss 3.4350531101226807 train acc 0.24386598857958602
epoch 1 batch id 1601 loss 2.602599859237671 train acc 0.23826905059337913
epoch 1 batch id 1801 loss 13.998085975646973 train acc 0.23349875069405884
epoch 1 batch id 2001 loss 13.515995979309082 train acc 0.23024425287356323
epoch 1 batch id 2201 loss 10.79781436920166 train acc 0.22743923216719672
epoch 1 batch id 2401 loss 9.103242874145508 train acc 0.22573927530195753
epoch 1 batch id 2601 loss 13.108366012573

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 1 test acc 0.332041852307062


  0%|          | 0/3395 [00:00<?, ?it/s]

...

epoch 5 test acc 0.1566244814871716


  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 6 batch id 1 loss 2.483792304992676 train acc 0.15625
epoch 6 batch id 201 loss 2.037726879119873 train acc 0.23149875621890548
epoch 6 batch id 401 loss 2.096045732498169 train acc 0.23145261845386533
epoch 6 batch id 601 loss 1.7864364385604858 train acc 0.23247712146422628
epoch 6 batch id 801 loss 1.9927068948745728 train acc 0.23431647940074907
epoch 6 batch id 1001 loss 1.6718286275863647 train acc 0.23944805194805194
epoch 6 batch id 1201 loss 1.7539863586425781 train acc 0.24180370524562864
epoch 6 batch id 1401 loss 1.9316866397857666 train acc 0.2434421841541756
epoch 6 batch id 1601 loss 1.7280086278915405 train acc 0.24611570893191756
epoch 6 batch id 1801 loss 1.8776930570602417 train acc 0.2484730705163798
epoch 6 batch id 2001 loss 1.7919366359710693 train acc 0.25181159420289856
epoch 6 batch id 2201 loss 1.7567132711410522 train acc 0.2553668786915039
epoch 6 batch id 2401 loss 1.6685898303985596 train acc 0.2593450645564348
epoch 6 batch id 2601 loss 1.736379384

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 6 test acc 0.332041852307062


학습률 0.05 , 에포크
(발산)

---

__학습률5e-5, 에포크3__

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 64    
batch_size = 32 #베이스라인 64
warmup_ratio = 0.1
num_epochs = 3  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))
    
# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,     
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [7]:
# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류 
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=0)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=0)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수(로딩속도)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=0)

# 모델 선언
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)  #로컬에서는 디폴트(0)으로 수정
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [8]:
train_accuarcy, test_accuarcy = [], []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    train_accuarcy.append(train_acc / (batch_id+1))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    test_accuarcy.append(test_acc / (batch_id+1))
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8545067310333252 train acc 0.15625
epoch 1 batch id 201 loss 1.527410626411438 train acc 0.28280472636815923
epoch 1 batch id 401 loss 1.1838306188583374 train acc 0.3892612219451372
epoch 1 batch id 601 loss 1.1383438110351562 train acc 0.45450291181364394
epoch 1 batch id 801 loss 1.0952427387237549 train acc 0.48892009987515606
epoch 1 batch id 1001 loss 1.0606321096420288 train acc 0.5161713286713286
epoch 1 batch id 1201 loss 1.0103175640106201 train acc 0.5336698584512906
epoch 1 batch id 1401 loss 1.1667410135269165 train acc 0.5482913990007138
epoch 1 batch id 1601 loss 0.9199232459068298 train acc 0.5585961898813242
epoch 1 batch id 1801 loss 1.1422905921936035 train acc 0.5668031649083842
epoch 1 batch id 2001 loss 1.1484204530715942 train acc 0.574197276361819
epoch 1 batch id 2201 loss 0.9804944396018982 train acc 0.5814260563380281
epoch 1 batch id 2401 loss 1.1562060117721558 train acc 0.5875026030820492
epoch 1 batch id 2601 loss 1.0403692722320

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 1 test acc 0.6740378706406513


  0%|          | 0/3395 [00:00<?, ?it/s]

...

epoch 2 test acc 0.6905646668715112


  0%|          | 0/3395 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.6888160705566406 train acc 0.71875
epoch 3 batch id 201 loss 0.7808974385261536 train acc 0.7492226368159204
epoch 3 batch id 401 loss 0.6058851480484009 train acc 0.742752493765586
epoch 3 batch id 601 loss 0.7424806952476501 train acc 0.7453202995008319
epoch 3 batch id 801 loss 0.7666810750961304 train acc 0.7468398876404494
epoch 3 batch id 1001 loss 0.6716791987419128 train acc 0.7514985014985015
epoch 3 batch id 1201 loss 0.7556205987930298 train acc 0.7546835970024979
epoch 3 batch id 1401 loss 0.6293813586235046 train acc 0.7579853675945754
epoch 3 batch id 1601 loss 0.5433908700942993 train acc 0.7600718301061836
epoch 3 batch id 1801 loss 0.6680213809013367 train acc 0.7614519711271516
epoch 3 batch id 2001 loss 0.7848539352416992 train acc 0.7631028235882059
epoch 3 batch id 2201 loss 0.629702091217041 train acc 0.7656746933212176
epoch 3 batch id 2401 loss 0.6801443696022034 train acc 0.7678701582673886
epoch 3 batch id 2601 loss 0.8584754467010498

  0%|          | 0/849 [00:00<?, ?it/s]

epoch 3 test acc 0.6939654068725355


In [9]:
train_accuarcy, test_accuarcy

([0.6073607020127639, 0.7052804369170349, 0.7728184830633285],
 [0.6740378706406513, 0.6905646668715112, 0.6939654068725355])

In [10]:
torch.save(model, 'model/kobert-v6.pt')

In [11]:
# 모델 사이즈 확인
import os

model_path = 'model/kobert-v6.pt'
size = os.path.getsize(model_path) / (1024*1024) # mb 단위
print(f"Model size: {size:.2f} MB")

Model size: 351.79 MB
